In [3]:
import numpy as np
from keras.datasets import fashion_mnist
def get_fa_mnist(flatten=True):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    if flatten:
        x_train = x_train.reshape(x_train.shape[0], 28*28)
        x_test = x_test.reshape(x_test.shape[0], 28*28)
    x_train = x_train.astype(np.float32) / 255.0
    x_test = x_test.astype(np.float32) / 255.0
    y_train = np.eye(10)[y_train]
    y_test = np.eye(10)[y_test]
    return (x_train, y_train), (x_test, y_test)

In [4]:
import numpy as np
class Activate:
    @staticmethod
    def leaky_relu(z, alpha=0.01):
        return np.where(z > 0, z, alpha*z)
    @staticmethod
    def leaky_relu_der(z, alpha=0.01):
        dz = np.ones_like(z)
        dz[z < 0] = alpha
        return dz
    @staticmethod
    def soft(z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [5]:
class FNN:
    def __init__(self, input_size, hidden_layers, output_size):
        np.random.seed(42)
        self.layers = [input_size] + hidden_layers + [output_size]
        self.weights = {}
        self.biases = {}
        for i in range(len(self.layers)-1):
            limit = np.sqrt(6 / (self.layers[i] + self.layers[i+1]))
            self.weights['W'+str(i+1)] = np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1]))
            self.biases['b'+str(i+1)] = np.zeros((1, self.layers[i+1]))
    def forw(self, X):
      self.cache = {'A0': X}
      L = len(self.layers)-1
      for l in range(1, L):
          Z = np.dot(self.cache['A'+str(l-1)], self.weights['W'+str(l)]) + self.biases['b'+str(l)]
          A = Activate.leaky_relu(Z)
          self.cache['Z'+str(l)] = Z
          self.cache['A'+str(l)] = A
      ZL = np.dot(self.cache['A'+str(L-1)], self.weights['W'+str(L)]) + self.biases['b'+str(L)]
      AL = Activate.soft(ZL)
      self.cache['Z'+str(L)] = ZL
      self.cache['A'+str(L)] = AL
      return AL
    def cross_ent(self,Y_pred, Y_true):
      m = Y_true.shape[0]
      return np.mean(-np.sum(Y_true * np.log(Y_pred + 1e-8), axis=1))
    def backw(self, Y_true, learning_rate=0.01):
      grads_w = {}
      grads_b = {}
      L = len(self.layers)-1
      m = Y_true.shape[0]
      dZ = self.cache['A'+str(L)] - Y_true
      grads_w['dW'+str(L)] = np.dot(self.cache['A'+str(L-1)].T, dZ)/m
      grads_b['db'+str(L)] = np.sum(dZ, axis=0, keepdims=True)/m
      dA_prev = np.dot(dZ, self.weights['W'+str(L)].T)
      for l in reversed(range(1, L)):
          dZ = dA_prev * Activate.leaky_relu_der(self.cache['Z'+str(l)])
          grads_w['dW'+str(l)] = np.dot(self.cache['A'+str(l-1)].T, dZ)/m
          grads_b['db'+str(l)] = np.sum(dZ, axis=0, keepdims=True)/m
          if l > 1:
              dA_prev = np.dot(dZ, self.weights['W'+str(l)].T)
      for l in range(1, L+1):
          self.weights['W'+str(l)] -= learning_rate * grads_w['dW'+str(l)]
          self.biases['b'+str(l)] -= learning_rate * grads_b['db'+str(l)]
    def train(self, X_train, Y_train, X_test, Y_test, lr=0.005, epochs=15):
      for epoch in range(epochs):
        Y_pred = self.forw(X_train)
        loss = self.cross_ent(Y_pred, Y_train)
        self.backw(Y_train, learning_rate=lr)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
      Y_pred_test = self.forw(X_test)
      accuracy = np.mean(np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1))
      print(f"Test Accuracy: {accuracy*100:.2f}%")

In [6]:
import numpy as np
class Activations:
    @staticmethod
    def relu(Z):
        return np.maximum(0, Z)

    @staticmethod
    def relu_der(Z):
        return (Z > 0).astype(float)

    @staticmethod
    def softmax(Z):
        exps = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)
def categorical_cross_entropy(pred, true):
    m = true.shape[0]
    return -np.sum(true * np.log(pred + 1e-8)) / m
class NeuralNet:
    def __init__(self, input_dim, hidden_layers, output_dim, optimizer='sgd', lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        np.random.seed(99)
        self.layers = [input_dim] + hidden_layers + [output_dim]
        self.weights = {}
        self.biases = {}
        self.opt = optimizer.lower()
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.time_step = 0
        for i in range(len(self.layers)-1):
            limit = np.sqrt(6 / (self.layers[i] + self.layers[i+1]))
            self.weights[f"W{i+1}"] = np.random.uniform(-limit, limit, (self.layers[i], self.layers[i+1]))
            self.biases[f"b{i+1}"] = np.zeros((1, self.layers[i+1]))
        self.velocity_w = {k: np.zeros_like(v) for k,v in self.weights.items()}
        self.velocity_b = {k: np.zeros_like(v) for k,v in self.biases.items()}
        self.sqr_w = {k: np.zeros_like(v) for k,v in self.weights.items()}
        self.sqr_b = {k: np.zeros_like(v) for k,v in self.biases.items()}
    def forward(self, X):
        store = {'A0': X}
        L = len(self.layers) - 1
        for l in range(1, L):
            Z = np.dot(store[f"A{l-1}"], self.weights[f"W{l}"]) + self.biases[f"b{l}"]
            A = Activations.relu(Z)
            store[f"Z{l}"], store[f"A{l}"] = Z, A

        ZL = np.dot(store[f"A{L-1}"], self.weights[f"W{L}"]) + self.biases[f"b{L}"]
        AL = Activations.softmax(ZL)
        store[f"Z{L}"], store[f"A{L}"] = ZL, AL
        return AL, store
    def backward(self, Y_pred, Y_true, store):
        grads = {}
        L = len(self.layers)-1
        m = Y_true.shape[0]
        dZ = Y_pred - Y_true
        grads[f"dW{L}"] = np.dot(store[f"A{L-1}"].T, dZ)/m
        grads[f"db{L}"] = np.sum(dZ, axis=0, keepdims=True)/m
        dA_prev = np.dot(dZ, self.weights[f"W{L}"].T)
        for l in reversed(range(1, L)):
            dZ = dA_prev * Activations.relu_der(store[f"Z{l}"])
            grads[f"dW{l}"] = np.dot(store[f"A{l-1}"].T, dZ)/m
            grads[f"db{l}"] = np.sum(dZ, axis=0, keepdims=True)/m
            if l > 1:
                dA_prev = np.dot(dZ, self.weights[f"W{l}"].T)
        return grads
    def update_params(self, grads):
        self.time_step += 1
        for l in range(1, len(self.layers)):
            Wk, bk = f"W{l}", f"b{l}"
            dW, db = grads[f"dW{l}"], grads[f"db{l}"]

            if self.opt == 'sgd':
                self.weights[Wk] -= self.lr * dW
                self.biases[bk] -= self.lr * db
            elif self.opt == 'momentum':
                self.velocity_w[Wk] = self.beta1*self.velocity_w[Wk] + (1-self.beta1)*dW
                self.velocity_b[bk] = self.beta1*self.velocity_b[bk] + (1-self.beta1)*db
                self.weights[Wk] -= self.lr * self.velocity_w[Wk]
                self.biases[bk] -= self.lr * self.velocity_b[bk]
            elif self.opt == 'nesterov':
              vW_prev = self.velocity_w[Wk]
              vb_prev = self.velocity_b[bk]
              self.velocity_w[Wk] = self.beta1 * self.velocity_w[Wk] - self.lr * dW
              self.velocity_b[bk] = self.beta1 * self.velocity_b[bk] - self.lr * db
              self.weights[Wk] += -self.beta1 * vW_prev + (1 + self.beta1) * self.velocity_w[Wk]
              self.biases[bk]  += -self.beta1 * vb_prev + (1 + self.beta1) * self.velocity_b[bk]
            elif self.opt == 'rmsprop':
                self.sqr_w[Wk] = self.beta2*self.sqr_w[Wk] + (1-self.beta2)*(dW**2)
                self.sqr_b[bk] = self.beta2*self.sqr_b[bk] + (1-self.beta2)*(db**2)
                self.weights[Wk] -= self.lr * dW / (np.sqrt(self.sqr_w[Wk])+self.eps)
                self.biases[bk] -= self.lr * db / (np.sqrt(self.sqr_b[bk])+self.eps)
            elif self.opt == 'adam':
                self.velocity_w[Wk] = self.beta1*self.velocity_w[Wk] + (1-self.beta1)*dW
                self.velocity_b[bk] = self.beta1*self.velocity_b[bk] + (1-self.beta1)*db
                self.sqr_w[Wk] = self.beta2*self.sqr_w[Wk] + (1-self.beta2)*(dW**2)
                self.sqr_b[bk] = self.beta2*self.sqr_b[bk] + (1-self.beta2)*(db**2)
                vW_corr = self.velocity_w[Wk]/(1-self.beta1**self.time_step)
                vb_corr = self.velocity_b[bk]/(1-self.beta1**self.time_step)
                sW_corr = self.sqr_w[Wk]/(1-self.beta2**self.time_step)
                sb_corr = self.sqr_b[bk]/(1-self.beta2**self.time_step)
                self.weights[Wk] -= self.lr * vW_corr / (np.sqrt(sW_corr)+self.eps)
                self.biases[bk] -= self.lr * vb_corr / (np.sqrt(sb_corr)+self.eps)
            elif self.opt == 'nadam':
                 self.velocity_w[Wk] = self.beta1*self.velocity_w[Wk] + (1-self.beta1)*dW
                 self.velocity_b[bk] = self.beta1*self.velocity_b[bk] + (1-self.beta1)*db
                 m_hat_b = self.velocity_b[bk] / (1 - self.beta1**self.time_step)
                 vW_hat = (self.velocity_w[Wk]*self.beta1 + (1-self.beta1)*dW) / (1-self.beta1**self.time_step)
                 vb_hat = (self.velocity_b[bk]*self.beta1 + (1-self.beta1)*db) / (1-self.beta1**self.time_step)
                 self.sqr_w[Wk] = self.beta2*self.sqr_w[Wk] + (1-self.beta2)*(dW**2)
                 self.sqr_b[bk] = self.beta2*self.sqr_b[bk] + (1-self.beta2)*(db**2)
                 sW_corr = self.sqr_w[Wk]/(1-self.beta2**self.time_step)
                 sb_corr = self.sqr_b[bk]/(1-self.beta2**self.time_step)
                 self.weights[Wk] -= self.lr * vW_hat / (np.sqrt(sW_corr)+self.eps)
                 self.biases[bk] -= self.lr * vb_hat / (np.sqrt(sb_corr)+self.eps)
    def train(self, X_train, Y_train, X_test, Y_test, epochs=20, batch_size=64):
        m = X_train.shape[0]
        for ep in range(epochs):
            perm = np.random.permutation(m)
            X_shuf, Y_shuf = X_train[perm], Y_train[perm]
            for i in range(0, m, batch_size):
                Xb, Yb = X_shuf[i:i+batch_size], Y_shuf[i:i+batch_size]
                Y_pred, store = self.forward(Xb)
                grads = self.backward(Y_pred, Yb, store)
                self.update_params(grads)
            Y_pred_train, _ = self.forward(X_train)
            train_loss = categorical_cross_entropy(Y_pred_train, Y_train)
            train_acc = np.mean(
            np.argmax(Y_pred_train, axis=1) == np.argmax(Y_train, axis=1)
            )
            Y_pred_test, _ = self.forward(X_test)
            val_loss = categorical_cross_entropy(Y_pred_test, Y_test)
            val_acc = np.mean(
                np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1)
            )
            wandb.log({
                "epoch": ep + 1,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            })

            print(
                f"Epoch {ep+1}/{epochs}, "
                f"Train Loss: {train_loss:.4f}, "
                f"Val Loss: {val_loss:.4f}, "
                f"Val Acc: {val_acc*100:.2f}%"
            )


In [7]:
(X_train, Y_train), (X_test, Y_test) = get_fa_mnist(flatten=True)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
import wandb

sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "test_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {
            "values": [10, 15, 20]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "learning_rate": {
            "values": [0.01, 0.005, 0.001]
        },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
        },
        "hidden_layers": {
            "values": [
                [64],
                [128],
                [128, 64],
                [256, 128]
            ]
        }
    }
}


In [9]:
def train_wandb():
    run = wandb.init()
    config = wandb.config

    model = NeuralNet(
        input_dim=784,
        hidden_layers=config.hidden_layers,
        output_dim=10,
        optimizer=config.optimizer,
        lr=config.learning_rate
    )

    model.train(
        X_train,
        Y_train,
        X_test,
        Y_test,
        epochs=config.epochs,
        batch_size=config.batch_size
    )

    # Final evaluation
    Y_pred_test, _ = model.forward(X_test)
    test_acc = np.mean(
        np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1)
    )

    wandb.log({
        "test_accuracy": test_acc
    } , commit=False)


In [10]:
sweep_id = wandb.sweep(
    sweep_config,
    project="fashion-mnist-q4"
)

wandb.agent(
    sweep_id,
    function=train_wandb,
    count=20
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Epoch 1/10, Train Loss: 0.3665, Val Loss: 0.4138, Val Acc: 85.09%
Epoch 2/10, Train Loss: 0.3390, Val Loss: 0.3972, Val Acc: 86.09%
Epoch 3/10, Train Loss: 0.2864, Val Loss: 0.3542, Val Acc: 87.25%
Epoch 4/10, Train Loss: 0.2827, Val Loss: 0.3612, Val Acc: 87.25%
Epoch 5/10, Train Loss: 0.2741, Val Loss: 0.3584, Val Acc: 87.19%
Epoch 6/10, Train Loss: 0.2369, Val Loss: 0.3290, Val Acc: 87.95%
Epoch 7/10, Train Loss: 0.2256, Val Loss: 0.3277, Val Acc: 88.15%
Epoch 8/10, Train Loss: 0.2252, Val Loss: 0.3292, Val Acc: 88.09%
Epoch 9/10, Train Loss: 0.2171, Val Loss: 0.3346, Val Acc: 88.32%
Epoch 10/10, Train Loss: 0.2133, Val Loss: 0.3465, Val Acc: 87.94%


epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
train_accuracy,▁▂▅▅▅▇▇▇██
train_loss,█▇▄▄▄▂▂▂▁▁
val_accuracy,▁▃▆▆▆▇███▇
val_loss,█▇▃▄▃▁▁▁▂▃
epoch,10
test_accuracy,0.8794
train_accuracy,0.92043
train_loss,0.2133
val_accuracy,0.8794


wandb: Agent Starting Run: vt6cyr9f with config:
wandb: 	batch_size: 32
wandb: 	epochs: 20
wandb: 	hidden_layers: [256, 128]
wandb: 	learning_rate: 0.001
wandb: 	optimizer: nadam
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


Epoch 1/20, Train Loss: 0.3665, Val Loss: 0.4138, Val Acc: 85.09%
Epoch 2/20, Train Loss: 0.3390, Val Loss: 0.3972, Val Acc: 86.09%
Epoch 3/20, Train Loss: 0.2864, Val Loss: 0.3542, Val Acc: 87.25%
Epoch 4/20, Train Loss: 0.2827, Val Loss: 0.3612, Val Acc: 87.25%
Epoch 5/20, Train Loss: 0.2741, Val Loss: 0.3584, Val Acc: 87.19%
Epoch 6/20, Train Loss: 0.2369, Val Loss: 0.3290, Val Acc: 87.95%
Epoch 7/20, Train Loss: 0.2256, Val Loss: 0.3277, Val Acc: 88.15%
Epoch 8/20, Train Loss: 0.2252, Val Loss: 0.3292, Val Acc: 88.09%
Epoch 9/20, Train Loss: 0.2171, Val Loss: 0.3346, Val Acc: 88.32%
Epoch 10/20, Train Loss: 0.2133, Val Loss: 0.3465, Val Acc: 87.94%
Epoch 11/20, Train Loss: 0.1966, Val Loss: 0.3382, Val Acc: 88.86%
Epoch 12/20, Train Loss: 0.1863, Val Loss: 0.3450, Val Acc: 88.80%
Epoch 13/20, Train Loss: 0.1802, Val Loss: 0.3432, Val Acc: 88.59%
Epoch 14/20, Train Loss: 0.1707, Val Loss: 0.3468, Val Acc: 89.17%
Epoch 15/20, Train Loss: 0.1653, Val Loss: 0.3429, Val Acc: 89.05%
Epoc

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_accuracy,▁
train_accuracy,▁▂▄▄▄▅▅▅▅▆▆▇▇▇▇▇▇█▇█
train_loss,█▇▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▂▁
val_accuracy,▁▃▅▅▅▆▆▆▆▆▇▇▇█████▇█
val_loss,█▇▃▄▃▁▁▁▂▃▂▂▂▃▂▂▅▄▅▅
epoch,20
test_accuracy,0.8915
train_accuracy,0.94605
train_loss,0.14264
val_accuracy,0.8915
